In [1]:
import os
import requests

from PIL import Image
from io import BytesIO
from jsonargparse import CLI

server_url = "https://8000-01hy7ksszs01eppcc804p9yjrh.cloudspaces.litng.ai/predict"


def send_image_to_server(prompt: str, image_path:str):
 
    image = Image.open(image_path).convert("RGB")

    # Convert the image to bytes
    buffered = BytesIO()
    image.save(buffered, format="JPEG")
    image_bytes = buffered.getvalue()

    headers = {"Authorization": "Bearer llmops"}
    response = requests.post(
        server_url,
        json={
            "image_bytes": image_bytes.hex(),  # Convert bytes to hex string for JSON serialization
            "prompt": prompt,
        },
    )

    # Print the response from the server
    if response.status_code == 200:
        out=response.json()
        return out["output"]
    else:
        print(
            "Failed to get response from the server, status code:", response.status_code
        )


In [3]:
image_path = "dog.jpg"
prompt = "what is in this image?"
send_image_to_server(image_path=image_path, prompt=prompt)

'A black and white dog and a gray and white cat are standing side by side on a yellow background. The dog has a black and white striped collar and its tongue is sticking out. The cat has green eyes and its ears are perked up. The dog is wearing a tie and the cat is looking at the camera. The two animals are together and looking at the viewer.'

### LLAMA 3 8b instruct (VLLM)

In [7]:
server_url="https://8001-01hy7ksszs01eppcc804p9yjrh.cloudspaces.litng.ai"

In [6]:
from openai import OpenAI

# Modify OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "EMPTY"
# openai_api_base = "http://localhost:8000/v1"
openai_api_base="https://8000-01hy7ksszs01eppcc804p9yjrh.cloudspaces.litng.ai/v1"

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=openai_api_key,
    base_url=openai_api_base,
)

models = client.models.list()
model = models.data[0].id

chat_completion = client.chat.completions.create(
    messages=[{
        "role": "system",
        "content": "You are a helpful assistant."
    }, {
        "role": "user",
        "content": "Who won the world series in 2020?"
    }, {
        "role":
        "assistant",
        "content":
        "The Los Angeles Dodgers won the World Series in 2020."
    }, {
        "role": "user",
        "content": "Where was it played?"
    }],
    model=model,
)

print(chat_completion.choices[0].message.content)

The 2020 World Series was played at the Globe Life Field in Arlington, Texas.


# using with litellm

In [20]:
import litellm 
messages=messages=[{
        "role": "system",
        "content": "You are a helpful assistant."
    }, {
        "role": "user",
        "content": "Who won the world series in 2020?"
    }, {
        "role":
        "assistant",
        "content":
        "The Los Angeles Dodgers won the World Series in 2020."
    }, {
        "role": "user",
        "content": "Where was it played?"
    }]

response = litellm.completion(
            model="openai/meta-llama/Meta-Llama-3-8B-Instruct", # pass the vllm model name
            messages=messages,
            api_key="sk-1234",
            api_base="https://8000-01hy7ksszs01eppcc804p9yjrh.cloudspaces.litng.ai/v1",
            temperature=0.2,
            max_tokens=80)

print(response['choices'][0]['message']['content'])

The 2020 World Series was played at Globe Life Field in Arlington, Texas, which is the home of the Texas Rangers. The series was played in a neutral site due to the COVID-19 pandemic, as the Dodgers and Tampa Bay Rays did not have a home-field advantage.


### LLAMA 3 8b instruct (TGI)

In [25]:
from openai import OpenAI

# init the client but point it to TGI
client = OpenAI(
    base_url="http://localhost:8080/v1",
    api_key="-"
)

chat_completion = client.chat.completions.create(
    model="tgi",
    messages=[
        {"role": "system", "content": "You are a helpful assistant." },
        {"role": "user", "content": "What is deep learning?"}
    ],
    stream=False
)

print(chat_completion.choices[0].message.content)

Deep learning is a subfield of artificial intelligence (AI) that involves the use of artificial neural networks to analyze and interpret data. These networks are composed of multiple layers of interconnected nodes or "neurons," hence the term "deep" learning.

In deep learning, these artificial neural networks are designed to mimic the structure and function of the human brain, in which complex patterns and relationships are learned through the interactions of multiple layers of neurons. This allows deep learning algorithms to learn and represent increasingly abstract and nuanced


## OLLAMA with LLAMA 3 8b

In [4]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',

    # required but ignored
    api_key='ollama',
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            'role': 'user',
            'content': 'what is the color of sky?',
        }
    ],
    model='llama3:8b',
)
print(chat_completion.choices[0].message.content)

What a great question!

The color of the sky can appear different depending on various factors, such as:

1. **Time of day**: During sunrise and sunset, the sky can take on hues of red, orange, pink, and purple due to the scattering of light by atmospheric particles.
2. **Atmospheric conditions**: On clear days, the sky appears blue because the shorter wavelengths of light (like blue and violet) are scattered more than the longer wavelengths (like red and orange). However, if there's pollution or dust in the air, the sky can appear hazy or gray.
3. **Altitude and atmospheric layers**: From a plane or high altitude, the sky can appear darker blue to almost black due to the reduced amount of scattering particles.
4. **Weather systems**: During thunderstorms or severe weather events, the sky can take on various shades of gray, brown, or even greenish hues.

So, what is the "standard" color of the sky? Well...

In ideal conditions, with a clear blue sky and no atmospheric distortion, the s